In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Predict hdlngth of possum with SVR

In [2]:
possum = pd.read_csv('possum.csv')

In [3]:
print(possum.head())

   case  site  Pop sex  age  hdlngth  skullw  totlngth  taill  footlgth  \
0     1     1  Vic   m  8.0     94.1    60.4      89.0   36.0      74.5   
1     2     1  Vic   f  6.0     92.5    57.6      91.5   36.5      72.5   
2     3     1  Vic   f  6.0     94.0    60.0      95.5   39.0      75.4   
3     4     1  Vic   f  6.0     93.2    57.1      92.0   38.0      76.1   
4     5     1  Vic   f  2.0     91.5    56.3      85.5   36.0      71.0   

   earconch   eye  chest  belly  
0      54.5  15.2   28.0   36.0  
1      51.2  16.0   28.5   33.0  
2      51.9  15.5   30.0   34.0  
3      52.2  15.2   28.0   34.0  
4      53.2  15.1   28.5   33.0  


In [4]:
print(possum.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   case      104 non-null    int64  
 1   site      104 non-null    int64  
 2   Pop       104 non-null    object 
 3   sex       104 non-null    object 
 4   age       102 non-null    float64
 5   hdlngth   104 non-null    float64
 6   skullw    104 non-null    float64
 7   totlngth  104 non-null    float64
 8   taill     104 non-null    float64
 9   footlgth  103 non-null    float64
 10  earconch  104 non-null    float64
 11  eye       104 non-null    float64
 12  chest     104 non-null    float64
 13  belly     104 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 11.5+ KB
None


### Drop column case

In [5]:
possum.drop(columns=['case'], inplace=True)

### Rearrange columns (move hdlngth to the right)

In [6]:
possum = possum.iloc[:, [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 5]]

In [7]:
print(possum.head())

   site  Pop sex  age  hdlngth  totlngth  taill  footlgth  earconch   eye  \
0     1  Vic   m  8.0     94.1      89.0   36.0      74.5      54.5  15.2   
1     1  Vic   f  6.0     92.5      91.5   36.5      72.5      51.2  16.0   
2     1  Vic   f  6.0     94.0      95.5   39.0      75.4      51.9  15.5   
3     1  Vic   f  6.0     93.2      92.0   38.0      76.1      52.2  15.2   
4     1  Vic   f  2.0     91.5      85.5   36.0      71.0      53.2  15.1   

   chest  belly  skullw  
0   28.0   36.0    60.4  
1   28.5   33.0    57.6  
2   30.0   34.0    60.0  
3   28.0   34.0    57.1  
4   28.5   33.0    56.3  


### Split the dataset into training and test set

In [8]:
possum_train, possum_test = train_test_split(possum, test_size=0.2, random_state=1)

In [9]:
print(possum_train)

    site    Pop sex  age  hdlngth  totlngth  taill  footlgth  earconch   eye  \
82     6  other   f  3.0     90.0      81.5   36.0      62.0      43.3  14.0   
66     5  other   m  4.0     93.3      85.0   36.5      64.7      44.1  16.5   
44     2    Vic   m  3.0     90.7      81.0   34.0      71.5      54.0  14.6   
42     2    Vic   f  2.0     90.0      81.0   32.0      72.0      49.4  13.4   
40     2    Vic   f  5.0     88.4      83.0   36.5       NaN      40.3  15.9   
..   ...    ...  ..  ...      ...       ...    ...       ...       ...   ...   
75     6  other   m  5.0     92.4      89.0   41.0      64.5      46.4  17.8   
9      1    Vic   f  6.0     91.8      89.5   37.5      70.9      53.4  14.4   
72     5  other   m  1.0     82.5      82.0   36.5      65.7      44.7  16.0   
12     1    Vic   m  5.0     95.1      89.5   36.0      71.0      49.8  15.8   
37     2    Vic   m  7.0     92.4      80.5   35.5      68.4      49.5  15.9   

    chest  belly  skullw  
82   25.0   

In [10]:
print(possum_test)

     site    Pop sex  age  hdlngth  totlngth  taill  footlgth  earconch   eye  \
96      7  other   m  3.0     91.0      86.0   38.0      63.8      46.0  14.5   
35      2    Vic   m  7.0     93.3      88.0   35.0      74.3      52.0  14.9   
81      6  other   f  4.0     86.0      82.0   36.5      60.7      42.9  15.4   
59      4  other   f  2.0     91.3      88.0   39.0      63.1      47.0  14.4   
38      2    Vic   f  1.0     84.7      75.0   34.0      68.7      53.4  13.0   
39      2    Vic   f  3.0     91.0      84.5   36.0      72.8      51.4  13.6   
53      4  other   m  7.0     96.9      91.5   43.0      71.3      46.0  17.5   
85      6  other   f  3.0     88.2      86.5   38.5      60.3      43.7  13.6   
31      1    Vic   f  4.0     94.3      94.0   39.0      74.8      52.0  14.9   
56      4  other   f  4.0     95.1      93.0   41.0      67.2      45.3  14.5   
98      7  other   f  3.0     93.3      86.5   38.5      64.8      43.8  14.0   
46      3  other   m  2.0   

### Handle missing values in train and test set

#### training set

In [11]:
print(possum_train.isna().sum())

site        0
Pop         0
sex         0
age         2
hdlngth     0
totlngth    0
taill       0
footlgth    1
earconch    0
eye         0
chest       0
belly       0
skullw      0
dtype: int64


#### calculate threshold for missing data

In [12]:
treshold_train = 0.05 * len(possum_train)
print(treshold_train)

4.15


As the number of missing values in both columns age and footlgth is below the treshold we can drop these observation.

#### drop missing data in training set

In [13]:
possum_train.dropna(axis=0, subset=['age', 'footlgth'], inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [14]:
print(possum_train.isna().sum())

site        0
Pop         0
sex         0
age         0
hdlngth     0
totlngth    0
taill       0
footlgth    0
earconch    0
eye         0
chest       0
belly       0
skullw      0
dtype: int64


#### test set

In [15]:
print(possum_test.isna().sum())

site        0
Pop         0
sex         0
age         0
hdlngth     0
totlngth    0
taill       0
footlgth    0
earconch    0
eye         0
chest       0
belly       0
skullw      0
dtype: int64


In test set we have no missing values to handle.

### Get X and y values for train and test set

In [16]:
X_train = possum_train.iloc[:, :-1].values
y_train = possum_train.iloc[:, -1].values

In [17]:
print(X_train)
print(y_train)

[[6 'other' 'f' 3.0 90.0 81.5 36.0 62.0 43.3 14.0 25.0 29.0]
 [5 'other' 'm' 4.0 93.3 85.0 36.5 64.7 44.1 16.5 27.5 29.5]
 [2 'Vic' 'm' 3.0 90.7 81.0 34.0 71.5 54.0 14.6 27.0 31.5]
 [2 'Vic' 'f' 2.0 90.0 81.0 32.0 72.0 49.4 13.4 29.0 31.0]
 [6 'other' 'm' 3.0 88.4 80.5 36.0 62.6 43.6 16.3 25.0 28.5]
 [7 'other' 'm' 2.0 98.5 93.0 41.5 71.7 46.8 15.0 26.0 36.0]
 [3 'other' 'm' 4.0 95.4 85.0 37.0 69.0 45.0 15.9 29.5 35.5]
 [2 'Vic' 'f' 2.0 89.3 82.5 35.0 71.2 52.0 13.6 28.0 31.5]
 [1 'Vic' 'f' 9.0 93.3 89.5 39.0 77.2 51.3 14.9 31.0 34.0]
 [1 'Vic' 'f' 6.0 94.0 95.5 39.0 75.4 51.9 15.5 30.0 34.0]
 [1 'Vic' 'm' 3.0 91.4 89.0 37.0 70.8 51.8 14.8 24.0 30.0]
 [5 'other' 'f' 7.0 91.9 87.0 38.0 65.4 44.1 13.0 27.0 34.0]
 [1 'Vic' 'f' 4.0 94.8 89.0 38.0 73.8 52.4 15.5 27.0 36.0]
 [1 'Vic' 'm' 4.0 93.8 87.0 34.5 73.2 53.0 15.3 27.0 30.0]
 [5 'other' 'f' 6.0 92.0 88.5 38.0 64.1 46.3 15.2 25.5 28.5]
 [7 'other' 'm' 1.0 89.5 81.5 36.5 66.0 46.8 14.8 23.0 27.0]
 [6 'other' 'm' 1.0 85.8 81.0 36.5 62.8 

In [18]:
X_test = possum_test.iloc[:, :-1].values
y_test = possum_test.iloc[:, -1].values

In [19]:
print(X_test)
print(y_test)

[[7 'other' 'm' 3.0 91.0 86.0 38.0 63.8 46.0 14.5 25.0 31.5]
 [2 'Vic' 'm' 7.0 93.3 88.0 35.0 74.3 52.0 14.9 25.5 36.0]
 [6 'other' 'f' 4.0 86.0 82.0 36.5 60.7 42.9 15.4 26.0 32.0]
 [4 'other' 'f' 2.0 91.3 88.0 39.0 63.1 47.0 14.4 26.0 30.0]
 [2 'Vic' 'f' 1.0 84.7 75.0 34.0 68.7 53.4 13.0 25.0 25.0]
 [2 'Vic' 'f' 3.0 91.0 84.5 36.0 72.8 51.4 13.6 27.0 30.0]
 [4 'other' 'm' 7.0 96.9 91.5 43.0 71.3 46.0 17.5 30.0 36.5]
 [6 'other' 'f' 3.0 88.2 86.5 38.5 60.3 43.7 13.6 26.0 31.0]
 [1 'Vic' 'f' 4.0 94.3 94.0 39.0 74.8 52.0 14.9 28.0 34.0]
 [4 'other' 'f' 4.0 95.1 93.0 41.0 67.2 45.3 14.5 31.0 39.0]
 [7 'other' 'f' 3.0 93.3 86.5 38.5 64.8 43.8 14.0 28.0 35.0]
 [3 'other' 'm' 2.0 90.1 89.0 37.5 66.0 45.5 15.0 25.0 33.0]
 [7 'other' 'm' 2.0 90.7 85.0 37.0 67.6 46.8 14.5 25.5 31.0]
 [4 'other' 'm' 2.0 102.5 96.0 40.0 73.2 44.5 14.7 32.0 36.0]
 [2 'Vic' 'm' 2.0 90.6 85.5 36.5 73.1 53.1 14.4 26.0 28.5]
 [7 'other' 'm' 6.0 97.7 84.5 35.0 64.4 46.2 14.4 29.0 30.5]
 [7 'other' 'f' 3.0 93.6 89.0 40.

#### Encode categorical data in train and test set

In [20]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(handle_unknown='ignore'), [0, 1, 2])], remainder='passthrough')
ct.fit(X_train)
X_train = ct.transform(X_train)

In [21]:
print(X_train)

[[0.0 0.0 0.0 ... 14.0 25.0 29.0]
 [0.0 0.0 0.0 ... 16.5 27.5 29.5]
 [0.0 1.0 0.0 ... 14.6 27.0 31.5]
 ...
 [0.0 0.0 0.0 ... 16.0 23.5 28.0]
 [1.0 0.0 0.0 ... 15.8 27.0 32.0]
 [0.0 1.0 0.0 ... 15.9 27.0 30.0]]


In [22]:
X_test = ct.transform(X_test)

In [23]:
print(X_test)

[[0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 3.0 91.0 86.0 38.0 63.8
  46.0 14.5 25.0 31.5]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 7.0 93.3 88.0 35.0 74.3
  52.0 14.9 25.5 36.0]
 [0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 4.0 86.0 82.0 36.5 60.7
  42.9 15.4 26.0 32.0]
 [0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 2.0 91.3 88.0 39.0 63.1
  47.0 14.4 26.0 30.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 1.0 84.7 75.0 34.0 68.7
  53.4 13.0 25.0 25.0]
 [0.0 1.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 3.0 91.0 84.5 36.0 72.8
  51.4 13.6 27.0 30.0]
 [0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 7.0 96.9 91.5 43.0 71.3
  46.0 17.5 30.0 36.5]
 [0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0 1.0 0.0 3.0 88.2 86.5 38.5 60.3
  43.7 13.6 26.0 31.0]
 [1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 0.0 4.0 94.3 94.0 39.0 74.8
  52.0 14.9 28.0 34.0]
 [0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 1.0 0.0 4.0 95.1 93.0 41.0 67.2
  45.3 14.5 31.0 39.0]
 [0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 1.0 1.0 0.0 3.0 93.3 86.5 38.5 64.8


### Scale train and test data

In [24]:
sc_X = StandardScaler()
sc_X.fit(X_train)
X_train = sc_X.transform(X_train)
X_test = sc_X.transform(X_test)
print(X_train)
print(X_test)

[[-0.79539491 -0.2847474  -0.2847474  ... -1.11563047 -1.00829134
  -1.43374571]
 [-0.79539491 -0.2847474  -0.2847474  ...  1.38298201  0.23268262
  -1.24064865]
 [-0.79539491  3.51188458 -0.2847474  ... -0.51596348 -0.01551217
  -0.46826038]
 ...
 [-0.79539491 -0.2847474  -0.2847474  ...  0.88325951 -1.75287571
  -1.81993984]
 [ 1.25723711 -0.2847474  -0.2847474  ...  0.68337051 -0.01551217
  -0.27516332]
 [-0.79539491  3.51188458 -0.2847474  ...  0.78331501 -0.01551217
  -1.04755158]]
[[-0.79539491 -0.2847474  -0.2847474  -0.19738551 -0.42008403 -0.37796447
   2.38047614 -0.92761259  0.92761259 -0.81649658  0.81649658 -0.46229895
  -0.53135492 -0.29447062  0.61716403 -1.09525432 -0.58797021 -0.61590798
  -1.00829134 -0.46826038]
 [-0.79539491  3.51188458 -0.2847474  -0.19738551 -0.42008403 -0.37796447
  -0.42008403  1.07803625 -1.07803625 -0.81649658  0.81649658  1.59236304
   0.14338149  0.19025468 -0.98412642  1.31441993  0.89508655 -0.21612998
  -0.76009655  1.2696132 ]
 [-0.79539

In [25]:
sc_y = StandardScaler()
sc_y.fit(np.reshape(y_train, (len(y_train), 1)))
y_train = sc_y.transform(np.reshape(y_train, (len(y_train), 1)))
print(y_train)

[[-1.03617668]
 [ 0.17862359]
 [-0.36483969]
 [-0.4927134 ]
 [-0.78042925]
 [ 1.16964487]
 [ 0.69011844]
 [-0.7164924 ]
 [ 0.05074988]
 [ 0.94586587]
 [-0.78042925]
 [-0.20499755]
 [-0.23696597]
 [-0.07712383]
 [-0.20499755]
 [-0.33287126]
 [-2.25097695]
 [ 0.33846573]
 [ 1.96885557]
 [ 1.42539229]
 [-1.93129267]
 [-0.68452397]
 [ 0.27452888]
 [ 0.81799216]
 [ 0.17862359]
 [-0.97223982]
 [-0.33287126]
 [ 0.30649731]
 [-0.9402714 ]
 [ 1.26555015]
 [-0.42877654]
 [ 0.33846573]
 [-0.81239768]
 [-0.58861868]
 [ 0.17862359]
 [ 0.46633945]
 [-0.23696597]
 [-0.39680812]
 [-1.61160839]
 [-0.20499755]
 [ 0.46633945]
 [ 0.01878145]
 [-0.9402714 ]
 [ 1.96885557]
 [ 3.6951507 ]
 [-0.23696597]
 [ 0.62618159]
 [ 1.07373958]
 [-0.14106069]
 [ 2.28853986]
 [-0.30090283]
 [ 0.30649731]
 [-0.17302912]
 [ 0.17862359]
 [-0.74846083]
 [ 0.17862359]
 [-0.46074497]
 [-0.78042925]
 [ 0.17862359]
 [-0.84436611]
 [-0.33287126]
 [-0.46074497]
 [-0.2689344 ]
 [-0.52468183]
 [ 0.33846573]
 [ 0.81799216]
 [ 0.62618

### Build the linear model

In [26]:
regressor = SVR(kernel='linear')
regressor.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR(kernel='linear')

### Predict the test set

In [27]:
y_pred = sc_y.inverse_transform(regressor.predict(X_test))

### Compare y_test with y_pred and calculate r2_score

In [28]:
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1))

[[56.42883162 53.1       ]
 [57.89799036 59.3       ]
 [53.21827001 54.        ]
 [56.21533794 57.7       ]
 [51.07323199 51.5       ]
 [55.67978377 55.        ]
 [61.46475591 63.        ]
 [53.4970267  53.2       ]
 [57.52559411 56.7       ]
 [59.69865556 59.4       ]
 [57.56045012 56.2       ]
 [56.57411471 54.8       ]
 [56.44333754 56.3       ]
 [63.45388558 62.8       ]
 [55.88497088 55.7       ]
 [59.66180704 58.4       ]
 [58.31972396 59.9       ]
 [56.65480976 56.        ]
 [56.87288248 55.7       ]
 [56.72848893 55.        ]
 [58.92085023 56.        ]]


In [29]:
print(r2_score(y_test, y_pred))

0.7597109029035543
